In [ ]:
import pandas as pd

# Load the dataset
file_path = '/content/full_dataset.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
df.head()


In [ ]:
# good one

from scipy.signal import resample
import numpy as np
import pandas as pd

def downsample_eeg_by_participant_and_epoch(data, original_fs, target_fs):
    # Group the data by 'participant_id' and 'epoch'
    grouped_data = data.groupby(['participant_id', 'epoch'])
    
    downsampled_data_list = []
    
    for (participant_id, epoch), group in grouped_data:
        # Calculate the number of samples for downsampling
        num_samples = int(len(group) * target_fs / original_fs)
        
        # Resample numeric columns (excluding 'participant_id' and 'epoch')
        numeric_cols = group.select_dtypes(include=[np.number]).columns
        
        downsampled_numeric_data = pd.DataFrame()
        for col in numeric_cols:
            downsampled_numeric_data[col] = resample(group[col], num_samples)
        
        # Custom rounding for the 'trigger' column
        def custom_round_trigger(val):
            val = np.round(val)
            return int(val)
        
        downsampled_numeric_data['trigger'] = downsampled_numeric_data['trigger'].apply(custom_round_trigger)
        
        # Add 'participant_id' and 'epoch' columns to the downsampled data
        downsampled_numeric_data['participant_id'] = participant_id
        downsampled_numeric_data['epoch'] = epoch
        
        downsampled_data_list.append(downsampled_numeric_data)
    
    # Merge all downsampled data from each participant and epoch
    downsampled_data = pd.concat(downsampled_data_list, ignore_index=True)
    
    return downsampled_data

# Load the dataset
file_path = '/content/epoched_filted_dataset_finally.csv'
df = pd.read_csv(file_path)

# Downsample the data for each participant and epoch and merge
downsampled_125Hz = downsample_eeg_by_participant_and_epoch(df, original_fs=250, target_fs=125)

# Verify the downsampling
verify_downsampling(df, downsampled_125Hz)


In [ ]:
from scipy.signal import resample
import numpy as np
import pandas as pd

def downsample_eeg_with_subject(data, original_fs, target_fs):
    numeric_cols = data.select_dtypes(include=[np.number]).columns.difference(['trigger'])
    non_numeric_cols = ['participant_id']
    
    # Calculate number of samples after downsampling
    num_samples = int(len(data) * target_fs / original_fs)
    
    # Resample numeric columns (excluding 'trigger')
    downsampled_numeric_data = pd.DataFrame()
    for col in numeric_cols:
        downsampled_numeric_data[col] = resample(data[col], num_samples)
    
    # Handle the 'trigger' column separately
    window_size = original_fs // target_fs  # Define the window size based on the downsampling ratio
    downsampled_trigger = []
    
    # Iterate through each window in the 'trigger' column
    for i in range(0, len(data), window_size):
        window_triggers = data['trigger'].iloc[i:i + window_size]
        
        # If there's at least one '1', keep it as '1', if not, check for '-1'
        if (window_triggers == 1).any():
            downsampled_trigger.append(1)
        elif (window_triggers == -1).any():
            downsampled_trigger.append(-1)
        else:
            downsampled_trigger.append(0)
    
    # Create a DataFrame for the downsampled 'trigger'
    downsampled_trigger_df = pd.DataFrame({'trigger': downsampled_trigger})
    
    # Process non-numeric columns (such as participant_id)
    downsampled_non_numeric_data = pd.DataFrame()
    for col in non_numeric_cols:
        downsampled_non_numeric_data[col] = [data[col].iloc[0]] * num_samples
        
    # Concatenate the downsampled numeric data, trigger column, and non-numeric columns
    downsampled_data = pd.concat([downsampled_numeric_data.reset_index(drop=True),
                                  downsampled_trigger_df.reset_index(drop=True),
                                  downsampled_non_numeric_data.reset_index(drop=True)], axis=1)
    
    return downsampled_data

# Example usage with original_fs = 250 Hz
original_fs = 250

# Downsample and preserve triggers
downsampled_128Hz = downsample_eeg_with_subject(df, original_fs, 125)

# Check the result
verify_downsampling(df, downsampled_128Hz)
